In [1]:
import streamlit as st
from sqlalchemy import text, exc
from uuid import uuid4
from datetime import datetime
import pandas as pd

METADATA_DB_URL = "sqlite:///session_metadata.sqlite"
thread_id = str(uuid4())

conn = st.connection(
        name="metadata_connector",
        type="sql",
        url=METADATA_DB_URL
    )

with conn.session as s:
    s.execute(text("""
        CREATE TABLE IF NOT EXISTS threads_metadata (
            thread_id TEXT PRIMARY KEY,
            name TEXT,
            created_at TEXT
            )
    """))
    s.commit()

def get_all_threads(conn):
    """
    Queries and returns all thread metadata, including error handling.
    """
    try:
        df = conn.query(
            "SELECT thread_id, name, created_at FROM threads_metadata ORDER BY created_at DESC",
            ttl=0  # Ensure the latest data is fetched
        )
        return df
    except exc.SQLAlchemyError as e:
        # Catch any general SQLAlchemy-related error
        st.error(f"**Query Error:** Could not retrieve the thread list. Please check the database connection and table structure. Details: {e}")
        return pd.DataFrame() # Return an empty DataFrame on failure
    
def create_new_thread(conn, thread_id, initial_name):
    """
    Creates a new thread record with the given ID and name, including error handling.
    """
    timestamp = datetime.now().isoformat()
    try:
        with conn.session as s:
            s.execute(
                text("INSERT INTO threads_metadata (thread_id, name, created_at) VALUES (:id, :name, :time)"),
                params={"id": thread_id, "name": initial_name, "time": timestamp}
            )
            s.commit()
        conn.reset() # Reset cache only upon successful commit
        return True # Success
    except exc.IntegrityError:
        # Catch errors like primary key violation (thread_id already exists)
        st.error(f"**Creation Failed:** Thread ID `{thread_id[:8]}...` already exists. Please use a unique ID.")
        return False
    except exc.SQLAlchemyError as e:
        st.error(f"**Database Error:** Could not create a new thread. Details: {e}")
        return False

def update_thread_name(conn, thread_id, new_name):
    """
    Updates the name of an existing thread, including error handling and existence check.
    """
    try:
        with conn.session as s:
            result = s.execute(
                text("UPDATE threads_metadata SET name = :new_name WHERE thread_id = :id"),
                params={"new_name": new_name, "id": thread_id}
            )
            s.commit()
        
        # Check if any rows were affected (i.e., if thread_id existed)
        if result.rowcount == 0:
            # Note: For SQLite, if the ID doesn't exist, it commits fine but rowcount is 0.
            st.warning(f"**Update Warning:** No thread found with ID `{thread_id[:8]}...` to update.")
            return False
            
        conn.reset()
        return True
    except exc.SQLAlchemyError as e:
        st.error(f"**Database Error:** Could not update the thread name. Details: {e}")
        return False

def delete_thread(conn, thread_id):
    """
    Deletes the thread record corresponding to the given thread_id, including error handling.
    """
    try:
        with conn.session as s:
            result = s.execute(
                text("DELETE FROM threads_metadata WHERE thread_id = :id"),
                params={"id": thread_id}
            )
            s.commit()
            
        # Check if any rows were affected (i.e., if thread_id existed)
        if result.rowcount == 0:
            st.warning(f"**Deletion Warning:** No thread found with ID `{thread_id[:8]}...` to delete.")
            return False

        conn.reset()
        return True
    except exc.SQLAlchemyError as e:
        st.error(f"**Database Error:** Could not delete the thread. Details: {e}")
        return False

print(delete_thread(conn, "927afd19-8998-47ba-a360-ce35944aeaf"))


2025-11-26 21:50:35.792 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:35.845 
  command:

    streamlit run /home/yueguobin/myCode/GNS3/gns3-copilot/venv/lib64/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-26 21:50:35.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:35.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:35.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:35.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:35.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21

False


In [2]:
print(get_all_threads(conn))


2025-11-26 21:50:40.834 No runtime found, using MemoryCacheStorageManager
2025-11-26 21:50:40.835 No runtime found, using MemoryCacheStorageManager
2025-11-26 21:50:40.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 21:50:40.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

                               thread_id                 name  \
0   57acf14a-4ef1-46db-8e9d-d4bf7226636e  Initial Thread Name   
1   3267a8b3-0ef3-403e-8190-f78a39727337  Updated Thread Name   
2   07acc21d-b48d-4907-88c5-434d7dabcf4b  Updated Thread Name   
3   f180d413-78c5-46d8-ba53-6937f37c5ef3  Updated Thread Name   
4   b78d7602-51e5-4b11-b012-56aa6bed6f40           New Thread   
5   1d921ccd-c71b-49c6-abd5-b9de7e4af3b9           New Thread   
6   45f6733d-8f8a-4f21-ac06-24e088b54fbb           New Thread   
7   f7181dd6-95d4-4d49-ac1f-0a2690bf3a1b           New Thread   
8   88da364d-ccf9-4c12-b28a-97f606793f3e           New Thread   
9   0037e540-3d35-414e-98d4-5d93087fd9bb           New Thread   
10  2f7a7817-e8ed-48e7-953c-dbaa7f263fa1           New Thread   
11  0298eb16-fde9-4842-b4fd-80ecffa6b938           New Thread   
12  5f186e1e-ced2-496a-968c-4a08d6c5793d           New Thread   
13  5cacea70-7e32-4e0a-b784-1e7226ff0108           New Thread   
14  a7a876f2-1078-43a8-9a

In [3]:
print(create_new_thread(conn, thread_id=str(uuid4()), initial_name="Initial Thread Name"))


True


In [4]:
print(update_thread_name(conn, thread_id="b00b16a5-485b-4627-a7f8-db4f3c84b1cc", new_name="安乐爱拉拉"))

True
